# Libs

In [1]:
import pandas as pd
import numpy as np
np.random.seed(1)
from functools import partial
import itertools
import optuna

import plotly.graph_objects as go
import plotly.express as px

from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS, NHITS
import sklearn.metrics as metrics

ponte = pd.read_pickle(r'Data\Data_Ponte_dos_Remedios.pkl')
del ponte['o3']
guarulhos = pd.read_pickle(r'Data\Data_Guarulhos.pkl')
guarulhos = guarulhos[['date','o3']]

data = ponte.merge(guarulhos, on='date', how='outer')
data.reset_index(drop=True)

import shutil
import pickle
from IPython.display import clear_output
import os
os.environ['NIXTLA_ID_AS_COL'] = '1'

from pytorch_lightning import Trainer
trainer = Trainer(
    max_steps=4,
    logger=False,
    enable_progress_bar=False,
    enable_model_summary=False  # Disable model summary
)

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="optuna")

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


# Gumodex

In [2]:
# USED WITH PLOTLY TO ELABORATE THE DESIGN ===========================================================
def main_layout(fig:go.Figure, width=700, height=600, x=None, y=None, title=None,
               x_range=None, y_range=None, paper_color='white', 
               customdata=None, hover_customdata='Info', 
               hover_x='x',hover_y='y', **kwargs) -> go.Figure:
    fig.layout = go.Layout(
        width=width,
        height=height,
        plot_bgcolor=paper_color,
        paper_bgcolor=paper_color,
        xaxis={'gridcolor':'#cccccc', 'linecolor':'black','title':x, 'range':x_range},
        yaxis={'gridcolor':'#cccccc', 'linecolor':'black','title':y, 'range':y_range},
        title={'text':title},
        **kwargs
    )
    if customdata == 'no':
        ...
    elif customdata is None:
        fig.update_traces(patch={
            'customdata':customdata, 'hovertemplate': hover_x + ': %{x}<br>' + hover_y + ': %{y}'
        })
    else:
        fig.update_traces(patch={
            'customdata':customdata,
            'hovertemplate': hover_x + ': %{x}<br>' + hover_y + ': %{y}<br>' + hover_customdata + ': %{customdata}<br>'
        })
    return fig
# ====================================================================================================
def main_subplot_layout(fig:go.Figure, width=1400, height=500, title=None, paper_color='white',
                        x=None, y=None, rows=1, cols=2, x_range=None, y_range=None,
                        customdata=None, hover_customdata='Info', 
                        hover_x='x',hover_y='y', **kwargs) -> go.Figure:
    fig.update_layout({
        'width':width,
        'height':height,
        'plot_bgcolor':paper_color,
        'paper_bgcolor':paper_color,
        'title':title,
        **kwargs
    })
    for xaxis in fig.select_xaxes():
        xaxis.update(
            showgrid=True,
            gridcolor='#CCCCCC',
            linecolor='black',
            title=x,
            range=x_range
        )
    for yaxis in fig.select_yaxes():
        yaxis.update(
            showgrid=True,
            gridcolor='#CCCCCC',
            linecolor='black',
            title=y,
            range=y_range
        )
    if customdata == 'no':
        ...
    elif customdata is None:
        fig.update_traces(patch={
            'customdata':customdata, 'hovertemplate': hover_x + ': %{x}<br>' + hover_y + ': %{y}'
        })
    else:
        fig.update_traces(patch={
            'customdata':customdata,
            'hovertemplate': hover_x + ': %{x}<br>' + hover_y + ': %{y}<br>' + hover_customdata + ': %{customdata}<br>'
        })
    return fig
# ====================================================================================================

# **Time Object**

In [3]:
class TimeObject:
    def __init__(self, df:pd.DataFrame, column:str, 
                 NAN_treatment_args:dict={'method':'from_derivatives'},
                 agg_freq:str=None) -> None:

        self.df = df[['date',column]]
        self.column = column
        self.time_serie = self.to_serie_()

        self.NAN_treatment_(**NAN_treatment_args)
        self.NIXTLA_treatment_()
        if agg_freq != None: 
            self.nixtla_df = self.nixtla_df.groupby(pd.Grouper(key='ds', freq=agg_freq)).agg({'y': 'mean'}).reset_index()
            self.nixtla_df.loc[:, ['unique_id']] = 1.0
        self.NIXTLA_train_test(split=7)

    def to_serie_(self) -> pd.Series:
        time_serie = self.df[self.column].fillna(np.nan)
        time_serie.index = pd.to_datetime(self.df['date'])

        full_index = pd.date_range(start=time_serie.index.min(), end=time_serie.index.max(), freq='D')
        time_serie = time_serie.reindex(full_index)
        return time_serie
    
    def NAN_treatment_(self, **kwargs) -> None:
        self.time_serie = self.time_serie.interpolate(**kwargs)
    
    def NIXTLA_treatment_(self) -> None:
        self.nixtla_df = pd.DataFrame()
        self.nixtla_df.loc[:, ['ds']] = pd.to_datetime(self.time_serie.index)
        self.nixtla_df.loc[:, ['y']] = self.time_serie.values
        self.nixtla_df.loc[:, ['unique_id']] = 1.0

    def plot(self) -> go.Figure:
        fig = go.Figure()
        fig.add_trace(trace=go.Scatter(
            x=self.time_serie.index, y=self.time_serie,
            marker=dict(color='#222222')
        ))
        return fig

    def NIXTLA_train_test(self, split:int=12):
        self.split = split
        self.Y_train = self.nixtla_df[self.nixtla_df.ds<self.nixtla_df['ds'].values[-split]]
        self.Y_test = self.nixtla_df[self.nixtla_df.ds>=self.nixtla_df['ds'].values[-split]].reset_index(drop=True)

    def metrics_(self, forecast_df:pd.DataFrame, method:str='NHITS'):

        def smape(y_true, y_pred):
            summation = 0
            for i in range(len(y_true)):
                summation += np.abs(y_true[i]-y_pred[i])/(np.abs(y_true[i]) + np.abs(y_pred[i]))
            return 200/(len(y_true)+1) * summation
        
        self.metrics = {}
        self.metrics['mae'] = np.round(metrics.mean_absolute_error(y_true=self.Y_test['y'], y_pred=forecast_df[method]),5)
        self.metrics['mape'] = np.round(100*metrics.mean_absolute_percentage_error(y_true=self.Y_test['y'], y_pred=forecast_df[method]),5)
        self.metrics['mse'] = np.round(metrics.mean_squared_error(y_true=self.Y_test['y'], y_pred=forecast_df[method]),5)
        self.metrics['max'] = np.round(metrics.max_error(y_true=self.Y_test['y'], y_pred=forecast_df[method]),5)
        self.metrics['smape'] = np.round(smape(y_true=self.Y_test['y'], y_pred=forecast_df[method]),5)
        return

    def plot_time_series(self):
        fig = go.Figure()
        fig.add_trace(trace=go.Scatter(
            x=self.Y_train['ds'], y=self.Y_train['y'],
            mode='lines', marker=go.scatter.Marker(
                color='black'
            ), name='Time Series'
        ))
        main_layout(fig=fig, width=1100, height=450, title='Time Series', x='time', y='AQI')
        return fig

    def plot_forecast(self, forecast_df:pd.DataFrame, confidence:int=90, method='NHITS', show:bool=True, show_metrics:bool=True):
        fig = go.Figure()

        fig.add_trace(trace=go.Scatter(
            x=self.Y_train['ds'], y=self.Y_train['y'],
            mode='lines', marker=go.scatter.Marker(
                color='black'
            ), name='train'
        ))
        
        fig.add_trace(trace=go.Scatter(
            x=self.Y_test['ds'], y=self.Y_test['y'],
            mode='lines', marker=go.scatter.Marker(
                color='skyblue'
            ), name='test'
        ))

        fig.add_trace(trace=go.Scatter(
            x=forecast_df['ds'], y=forecast_df[f'{method}'],
            mode='lines', marker=go.scatter.Marker(
                color='orange'
            ), name=method
        ))

        try:
            fig.add_trace(go.Scatter(
                x=forecast_df['ds'], y=forecast_df[f'{method}-lo-{confidence}'],
                mode='lines', line=dict(width=0), fill='tonexty',
                fillcolor='rgba(255, 165, 0, 0)',
                showlegend=False
            ))

            fig.add_trace(go.Scatter(
                x=forecast_df['ds'], y=forecast_df[f'{method}-hi-{confidence}'],
                mode='lines', line=dict(width=0), fill='tonexty',
                fillcolor='rgba(255, 165, 0, 0.2)',
                name=f'confidence: {confidence}%'
            ))
        except: ...

        main_layout(fig=fig, width=1100, height=450, title='Forecast', x='time', y='AQI')

        if show:
            fig.show()
        if show_metrics:
            self.metrics_(forecast_df, method=method)
            for key, metric in self.metrics.items():
                print(f'{key}: {metric}')
        
        return fig

# **Optuna**

### **N-HiTS**

In [ ]:
# Define the objective function
def objective(trial, pollutant, horizon):
    # Hyperparameter search space
    input_size = trial.suggest_int('input_size', 4, 156, step=1)
    n_stacks = trial.suggest_int('n_stacks', 3, 7, step=1)
    n_blocks = trial.suggest_int('n_blocks', 1, 7, step=1)
    max_steps = trial.suggest_int('max_steps', 1, 700, step=1)
    local_scalar_type = trial.suggest_categorical('local_scalar_type', [None, 'standard', 'boxcox', 'minmax'])
    n_pool_kernel_size = trial.suggest_categorical('n_pool_kernel_size', [list(combination) for combination in list(itertools.product([1, 2, 3], repeat=3))])
    n_freq_downsample = trial.suggest_categorical('n_freq_downsample', [list(combination) for combination in list(itertools.product([1, 4, 12, 52], repeat=3))])

    mape = []
    smape = []
    max = []
    mae = []
    mse = []
    # Split for cross validation
    for split in [1, 90]:
        print(f'\nPollutant = {pollutant} \nh = {horizon} \nTrial = {trial.number+1}\n')
        # Instantiate TimeObject and prepare training data
        obj = TimeObject(df=data[:-split], column=pollutant, agg_freq='W')
        obj.NIXTLA_train_test(split=horizon)

        # Define the model
        model = NHITS(
            h=horizon,
            input_size=input_size,
            stack_types=n_stacks*['identity'],
            n_freq_downsample=n_freq_downsample+(n_stacks-len(n_freq_downsample))*[1],
            n_blocks=n_stacks*[n_blocks],
            n_pool_kernel_size=(n_stacks-len(n_pool_kernel_size))*[1]+n_pool_kernel_size,
            pooling_mode="MaxPool1d",
            activation="ReLU",
            interpolation_mode='linear',
            max_steps=max_steps,
            val_check_steps=12,
        )

        # Initialize NeuralForecast and fit the model
        fcst = NeuralForecast(
            models=[model],
            freq='W',
            local_scaler_type=local_scalar_type
        )
        fcst.fit(df=obj.Y_train, verbose=False)
        prediction = fcst.predict(df=obj.Y_train, verbose=False)

        # Evaluate metrics
        obj.metrics_(forecast_df=prediction, method='NHITS')
        mape.append(obj.metrics['mape'])
        smape.append(obj.metrics['smape'])
        max.append(obj.metrics['max'])
        mae.append(obj.metrics['mae'])
        mse.append(obj.metrics['mse'])
        
        clear_output(wait=True)

    directory_path = "lightning_logs"
    if os.path.exists(directory_path):
        shutil.rmtree(directory_path)

    mape = np.mean(mape)
    smape = np.mean(smape)
    max = np.mean(max)
    mae = np.mean(mae)
    mse = np.mean(mse)

    # Collect the results
    results.append({
        'poll': pollutant,
        'freq': 'W',
        'split': split,
        'h': horizon,
        'input_size': input_size,
        'n_stacks': n_stacks,
        'n_blocks': n_blocks,
        'max_steps': max_steps,
        'local_scalar_type': local_scalar_type,
        'n_pool_kernel_size': n_pool_kernel_size,
        'n_freq_downsample': n_freq_downsample,
        'mape': mape,
        'smape': smape,
        'max': max,
        'mae': mae,
        'mse': mse,
    })

    # The objective for Optuna is to minimize the MAE (or maximize a metric)
    return smape, max, mae  # You can change this to any metric you want to optimize

for pollutant in data[['pm25','co','no2','o3','pm10']]:
    for h in [12, 52, 104]:
        # Initialize the results list
        results = []
        # Define the optimization study_nhits
        study_nhits = optuna.create_study(directions=['minimize','minimize','minimize'])  # Minimize the MAE

        # Run the optimization with the number of trials you want
        study_nhits.optimize(partial(objective, pollutant=pollutant, horizon=h), n_trials=50)

        clear_output(wait=True)
        NHITS_W = pd.DataFrame(results)
        NHITS_W.to_pickle(fr'Results\NHITS_{h}W_Df_{pollutant}.pkl')
        with open(fr"Results\NHITS_{h}W_Study_{pollutant}.pkl", "wb") as f:
            pickle.dump(study_nhits, f)

### **N-BEATS**

In [ ]:
# Define the objective function
def objective(trial, pollutant, horizon):
    # Hyperparameter search space
    input_size = trial.suggest_int('input_size', 4, 156, step=1)
    n_stacks = trial.suggest_int('n_stacks', 2, 7, step=1)
    n_blocks = trial.suggest_int('n_blocks', 1, 5, step=1)
    max_steps = trial.suggest_int('max_steps', 1, 700, step=1)
    local_scalar_type = trial.suggest_categorical('local_scalar_type', [None, 'standard', 'boxcox', 'minmax'])
    interpretability = trial.suggest_categorical('interpretability', [['seasonality','trend'],['seasonality','identity'],['trend','identity'],['identity','identity']])

    mape = []
    smape = []
    max = []
    mae = []
    mse = []
    # Split for cross validation
    for split in [1, 90]:
        # Instantiate TimeObject and prepare training data
        obj = TimeObject(df=data[:-split], column=pollutant, agg_freq='W')
        obj.NIXTLA_train_test(split=horizon)

        # Define the model
        model = NBEATS(
            h=horizon,
            input_size=input_size,
            stack_types=interpretability+(n_stacks-len(interpretability))*['identity'],
            n_blocks=n_stacks * [n_blocks],
            max_steps=max_steps,
            val_check_steps=12,
            learning_rate=1e-3
        )

        # Initialize NeuralForecast and fit the model
        fcst = NeuralForecast(
            models=[model],
            freq='W',
            local_scaler_type=local_scalar_type
        )
        fcst.fit(df=obj.Y_train, verbose=False)
        prediction = fcst.predict(df=obj.Y_train, verbose=False)

        # Evaluate metrics
        obj.metrics_(forecast_df=prediction, method='NBEATS')
        mape.append(obj.metrics['mape'])
        smape.append(obj.metrics['smape'])
        max.append(obj.metrics['max'])
        mae.append(obj.metrics['mae'])
        mse.append(obj.metrics['mse'])
            
        clear_output(wait=True)

        directory_path = "lightning_logs"
        if os.path.exists(directory_path):
            shutil.rmtree(directory_path)

    mape = np.mean(mape)
    smape = np.mean(smape)
    max = np.mean(max)
    mae = np.mean(mae)
    mse = np.mean(mse)

    # Collect the results
    results.append({
        'pollutant': pollutant,
        'freq': 'W',
        'split': split,
        'h': horizon,
        'input_size': input_size,
        'n_stacks': n_stacks,
        'n_blocks': n_blocks,
        'max_steps': max_steps,
        'local_scalar_type': local_scalar_type,
        'interpretability': interpretability,
        'mape': mape,
        'smape': smape,
        'max': max,
        'mae': mae,
        'mse': mse,
    })

    # The objective for Optuna is to minimize the MAE (or maximize a metric)
    return smape  # You can change this to any metric you want to optimize

for pollutant in data[['pm10','pm25','co','no2','o3']]:
    for h in [12, 52, 104]:
        # Initialize the results list
        results = []
        # Define the optimization study_nbeats
        study_nbeats = optuna.create_study(direction='minimize')  # Minimize the MAE

        # Run the optimization with the number of trials you want
        study_nbeats.optimize(partial(objective, pollutant=pollutant, horizon=h), n_trials=50)

        clear_output(wait=True)
        NBEATS_W = pd.DataFrame(results)
        NBEATS_W.to_pickle(fr'Results\NBEATS_{h}W_Df_{pollutant}.pkl')
        with open(fr"Results\NBEATS_{h}W_Study_{pollutant}.pkl", "wb") as f:
            pickle.dump(study_nbeats, f)

[I 2024-12-09 12:42:20,704] Trial 2 finished with value: 40.79736666666667 and parameters: {'input_size': 99, 'n_stacks': 7, 'n_blocks': 1, 'max_steps': 1, 'local_scalar_type': None, 'interpretability': ['identity', 'identity']}. Best is trial 1 with value: 24.015110000000004.


### Optuna Graphs

In [ ]:
fig = optuna.visualization.plot_contour(
    study_nbeats,
    target=lambda t: t.values[0], target_name="smape",
    params=['input_size','n_stacks','max_steps']
)
fig.update_layout({'height':600, 'width':800, 'plot_bgcolor':'white'})
fig.show()

fig = optuna.visualization.plot_contour(
    study_nbeats,
    target=lambda t: t.values[0], target_name="mae",
    params=['input_size','n_stacks','max_steps']
)
fig.update_layout({'height':600, 'width':800, 'plot_bgcolor':'white'})
fig.show()

# **Statistical Methods**

In [ ]:
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA, AutoCES, AutoETS, AutoTheta

results_stats = []
for pollutant in ['co','pm10','pm25','o3','no2']:
    for h in [12, 52, 104]:
        obj = TimeObject(data, pollutant, agg_freq='W')
        obj.NIXTLA_train_test(split=h)

        season_length = 52 # Monthly data 
        horizon = len(obj.Y_train) # number of predictions

        models = [
            AutoARIMA(season_length=season_length, alias='AutoARIMA'),
            AutoCES(season_length=season_length, model='Z', alias='AutoCES-Z'),
            AutoCES(season_length=season_length, model='S', alias='AutoCES-S'),
            AutoCES(season_length=season_length, model='P', alias='AutoCES-P'),
            AutoCES(season_length=season_length, model='N', alias='AutoCES-N'),
            AutoTheta(season_length=season_length, decomposition_type="multiplicative", alias='AutoTheta-Multi'),
            AutoTheta(season_length=season_length, decomposition_type="additive", alias='AutoTheta-Add'),
        ]
        ets_combinations = [
            f"{e}{t}{s}" for e in ['Z',]
                         for t in ['Z',] 
                         for s in ['Z', 'A', 'M', 'N'] 
        ]
        models = models + [
            AutoETS(season_length=season_length, model=ets, alias=f'AutoETS-{ets}')
            for ets in ets_combinations
        ]

        frct = StatsForecast(models=models, freq='W')
        frct.fit(df=obj.Y_train)
        predicted = frct.predict(h=h)

        columns = predicted.columns
        columns = columns[(columns != 'ds') & (columns != 'unique_id')]

        for method in columns:
            obj.metrics_(predicted, method=method)
            results_stats.append({
                'pollutant': pollutant,
                'method':method,
                'freq': 'W',
                'h': h,
                'mape': obj.metrics['mape'],
                'smape': obj.metrics['smape'],
                'max': obj.metrics['max'],
                'mae': obj.metrics['mae'],
                'mse': obj.metrics['mse'],
            })
            # obj.plot_forecast(predicted, method=method)

results_stats = pd.DataFrame(results_stats)
results_stats.to_pickle(r'Results\StatisticalMethods_W_Df.pkl')